# Imports

In [221]:
import numpy as np
import pandas as pd 

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

from xgboost import XGBRegressor
import xgboost as xgb

import matplotlib.pyplot as plt

import re


# Loading Data

In [222]:
train_path = "data/train.csv"
test_path = "data/test_x.csv"

train_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

train_raw = train_raw.dropna(subset=['Degerlendirme Puani'])

train_y_raw = train_raw['Degerlendirme Puani']
train_X_raw = train_raw

test_X_raw = test_raw

C:\Users\Alperen\AppData\Local\Temp\ipykernel_2444\1623258368.py:4: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  train_raw = pd.read_csv(train_path)


# Preprocessing

## Features

In [223]:
dropped = ['Lise Adi', 'Lise Adi Diger', 'Lise Sehir', 'Lise Bolumu', 'Lise Bolum Diger', 'Lise Mezuniyet Notu', 
           'Burs Aldigi Baska Kurum', 'Daha Once Baska Bir Universiteden Mezun Olmus', 'Baska Kurumdan Aldigi Burs Miktari', 'Anne Egitim Durumu', 'Baba Egitim Durumu',
           'Uye Oldugunuz Kulubun Ismi', "Hangi STK'nin Uyesisiniz?",
           'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite', 'id']
train_X_raw = train_X_raw.drop(dropped, axis=1)
test_X_raw = test_X_raw.drop(dropped, axis=1)

In [224]:
tr_chars = {'İ': 'I', 'Ü': 'U', 'Ö': 'O', 'Ğ': 'G', 'Ş': 'S', 'Ç': 'C'}

### Doğum tarihi ve Yaş

In [225]:
def tarih_formatla(df):
    def extract_year(date_str):
        if not isinstance(date_str, str):
            return None
        
        # Farklı tarih formatları için regex pattern'leri
        patterns = [
            r'(\d{4})[-/.](\d{1,2})[-/.](\d{1,2})',  # YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD
            r'(\d{1,2})[-/.](\d{1,2})[-/.](\d{4})',  # DD-MM-YYYY, DD/MM/YYYY, DD.MM.YYYY
            r'(\d{2})[-/.](\d{1,2})[-/.](\d{1,2})',  # YY-MM-DD, YY/MM/DD, YY.MM.DD
            r'(\d{1,2})[-/.](\d{1,2})[-/.](\d{2})'   # DD-MM-YY, DD/MM/YY, DD.MM.YY
        ]
        
        for pattern in patterns:
            match = re.search(pattern, date_str)
            if match:
                groups = match.groups()
                if len(groups[0]) == 4:  # YYYY MM DD formatı
                    year = int(groups[0])
                elif len(groups[2]) == 4:  # DD MM YYYY formatı
                    year = int(groups[2])
                else:  # YY-MM-DD veya DD-MM-YY formatı
                    year = int(groups[0] if len(groups[0]) == 2 else groups[2])
                    year += 1900 if year > 24 else 2000
                return year
        
        return np.nan
    
    def yas_sinirla(entry):
        dogum_tarihi = entry['Dogum Tarihi']
        basvurudugu_yil = entry['Basvuru Yili']
        return dogum_tarihi if 1900 <= dogum_tarihi <= basvurudugu_yil - 18 else np.nan
    
    # Yılları çıkar
    df['Dogum Tarihi'] = df["Dogum Tarihi"].apply(extract_year)
    df['Dogum Tarihi'] = df['Dogum Tarihi'].apply(lambda x: x if 1900 <= x <= 2007 else np.nan)
    df['yas'] = 2024 - df['Dogum Tarihi']
    # Geçersiz yılları ortalama ile doldur
    mean_year = df['yas'].mean()
    df['yas'] = df['yas'].fillna(round(mean_year)).astype(int)
    
    
    return df

train_X_raw['Dogum Tarihi'] = train_X_raw['Dogum Tarihi'].replace('1/1/70 2:00', np.nan)
# Başvuru yılı ve Doğum tarihi feature ı kullanarak yaş feature ı üretildi.
train_X_raw = tarih_formatla(train_X_raw)
test_X_raw = tarih_formatla(test_X_raw)

train_X_raw.drop("Dogum Tarihi", axis=1, inplace=True)
test_X_raw.drop("Dogum Tarihi", axis=1, inplace=True)

train_X_raw['Basvuru Yili'] = train_X_raw['Basvuru Yili'] - 2014
test_X_raw['Basvuru Yili'] = test_X_raw['Basvuru Yili'] - 2014
#test_X_raw["Dogum Tarihi"].value_counts(dropna=False).to_csv("tsetf.csv")
#train_X_raw["yas"].value_counts()

### Cinsiyet(One Hot)

In [226]:
train_X_raw['Cinsiyet'] = train_X_raw['Cinsiyet'].str.lower()
train_X_raw['Cinsiyet'] = train_X_raw['Cinsiyet'].replace(to_replace=[np.nan, 'belirtmek istemiyorum'], value='diger')
test_X_raw['Cinsiyet'] = test_X_raw['Cinsiyet'].str.lower()
test_X_raw['Cinsiyet'] = test_X_raw['Cinsiyet'].replace(to_replace=[np.nan, 'belirtmek istemiyorum'], value='diger')

### Doğum Yeri(Target encoding) ve İkametgah Şehri(Target encoding)

In [227]:
from rapidfuzz import process

sehirler=["Adana", "Adıyaman", "Afyon", "Ağrı", "Amasya", "Ankara", "Antalya", "Artvin", "Aydın", "Balıkesir", "Bilecik", "Bingöl", "Bitlis", "Bolu", "Burdur", "Bursa", "Çanakkale", "Çankırı", "Çorum", "Denizli", "Diyarbakır", "Edirne", "Elazığ", "Erzincan", "Erzurum", "Eskişehir", "Gaziantep", "Giresun", "Gümüşhane", "Hakkari", "Hatay", "Isparta", "İçel (Mersin)", "İstanbul", "İzmir", "Kars", "Kastamonu", "Kayseri", "Kırklareli", "Kırşehir", "Kocaeli", "Konya", "Kütahya", "Malatya", "Manisa", "Kahramanmaraş", "Mardin", "Muğla", "Muş", "Nevşehir", "Niğde", "Ordu", "Rize", "Sakarya", "Samsun", "Siirt", "Sinop", "Sivas", "Tekirdağ", "Tokat", "Trabzon", "Tunceli", "Şanlıurfa", "Uşak", "Van", "Yozgat", "Zonguldak", "Aksaray", "Bayburt", "Karaman", "Kırıkkale", "Batman", "Şırnak", "Bartın", "Ardahan", "Iğdır", "Yalova", "Karabük", "Kilis", "Osmaniye", "Düzce"
"KKTC"]

def temizle_metin(metin):
    if pd.isna(metin):
        return metin
    metin = str(metin).lower().strip()
    metin = re.sub(r'[^\w\s]', '', metin)
    return metin
# EŞİK DEĞİŞTİRİLEBİLİR
def en_yakin_eslesme(metin, secenekler, esik=70):
    if pd.isna(metin):
        return np.nan
    en_iyi_eslesme = process.extractOne(metin, secenekler)
    if en_iyi_eslesme[1] >= esik:
        return en_iyi_eslesme[0]
    return np.nan

def sehir_adi_temizle_ve_standartlastir(df, column_name):
    df[column_name] = df[column_name].apply(temizle_metin)
    df[column_name] = df[column_name].apply(lambda x: en_yakin_eslesme(x, sehirler))
    
    return df

train_X_raw = sehir_adi_temizle_ve_standartlastir(train_X_raw, 'Dogum Yeri')
train_X_raw['Dogum Yeri'] = train_X_raw['Dogum Yeri'].fillna('diger')
test_X_raw = sehir_adi_temizle_ve_standartlastir(test_X_raw, 'Dogum Yeri')
test_X_raw['Dogum Yeri'] = test_X_raw['Dogum Yeri'].fillna('diger')


train_X_raw = sehir_adi_temizle_ve_standartlastir(train_X_raw, 'Ikametgah Sehri')
train_X_raw['Ikametgah Sehri'] = train_X_raw['Ikametgah Sehri'].fillna('diger')
test_X_raw = sehir_adi_temizle_ve_standartlastir(test_X_raw, 'Ikametgah Sehri')
test_X_raw['Ikametgah Sehri'] = test_X_raw['Ikametgah Sehri'].fillna('diger')

target_mean = train_X_raw.groupby('Dogum Yeri')['Degerlendirme Puani'].mean()

train_X_raw['encoded_dogum_yeri'] = train_X_raw['Dogum Yeri'].map(target_mean)
test_X_raw['encoded_dogum_yeri'] = test_X_raw['Dogum Yeri'].map(target_mean)

target_mean = train_X_raw.groupby('Ikametgah Sehri')['Degerlendirme Puani'].mean()

train_X_raw['encoded_ikametgah_sehri'] = train_X_raw['Ikametgah Sehri'].map(target_mean)
test_X_raw['encoded_ikametgah_sehri'] = test_X_raw['Ikametgah Sehri'].map(target_mean)

global_mean = train_X_raw['Degerlendirme Puani'].mean()
test_X_raw['encoded_dogum_yeri'] = test_X_raw['encoded_dogum_yeri'].fillna(global_mean)
test_X_raw['encoded_ikametgah_sehri'] = test_X_raw['encoded_ikametgah_sehri'].fillna(global_mean)

#frequency = train_X_raw['Dogum Yeri'].value_counts()
#train_X_raw["encoded_dogum_yeri"] = train_X_raw['Dogum Yeri'].map(frequency)
#frequency = test_X_raw['Dogum Yeri'].value_counts()
#test_X_raw["encoded_dogum_yeri"] = test_X_raw['Dogum Yeri'].map(frequency)
#
#frequency = train_X_raw['Ikametgah Sehri'].value_counts()
#train_X_raw["encoded_ikametgah_sehri"] = train_X_raw['Ikametgah Sehri'].map(frequency)
#frequency = test_X_raw['Ikametgah Sehri'].value_counts()
#test_X_raw["encoded_ikametgah_sehri"] = test_X_raw['Ikametgah Sehri'].map(frequency)

train_X_raw.drop('Dogum Yeri', axis=1, inplace=True)
test_X_raw.drop('Dogum Yeri', axis=1, inplace=True)
train_X_raw.drop('encoded_dogum_yeri', axis=1, inplace=True)
test_X_raw.drop('encoded_dogum_yeri', axis=1, inplace=True)

train_X_raw.drop('Ikametgah Sehri', axis=1, inplace=True)
test_X_raw.drop('Ikametgah Sehri', axis=1, inplace=True)

### Üniversite Adı(Target encoding)

In [228]:
train_X_raw['Universite Adi'] = train_X_raw['Universite Adi'].fillna('DIGER')
train_X_raw['Universite Adi'] = train_X_raw['Universite Adi'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Universite Adi'] = train_X_raw['Universite Adi'].apply(lambda x: str(x).replace(tr_char, en_char))
    test_X_raw['Universite Adi'] = test_X_raw['Universite Adi'].apply(lambda x: str(x).replace(tr_char, en_char))

test_X_raw['Universite Adi'] = test_X_raw['Universite Adi'].fillna('DIGER')
test_X_raw['Universite Adi'] = test_X_raw['Universite Adi'].str.upper()

target_mean = train_X_raw.groupby('Universite Adi')['Degerlendirme Puani'].mean()

train_X_raw['encoded_universite_adi'] = train_X_raw['Universite Adi'].map(target_mean)
test_X_raw['encoded_universite_adi'] = test_X_raw['Universite Adi'].map(target_mean)

global_mean = train_X_raw['Degerlendirme Puani'].mean()
test_X_raw['encoded_universite_adi'] = test_X_raw['encoded_universite_adi'].fillna(global_mean)
#frequency = train_X_raw['Universite Adi'].value_counts()
#train_X_raw["encoded_universite_adi"] = train_X_raw['Universite Adi'].map(frequency)
#
#frequency = test_X_raw['Universite Adi'].value_counts()
#test_X_raw["encoded_universite_adi"] = test_X_raw['Universite Adi'].map(frequency)

train_X_raw.drop('Universite Adi', axis=1, inplace=True)
test_X_raw.drop('Universite Adi', axis=1, inplace=True)



C:\Users\Alperen\AppData\Local\Temp\ipykernel_2444\2123771137.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_X_raw['encoded_universite_adi'].fillna(global_mean, inplace=True)


### Üniversite Türü(One Hot)

In [229]:
train_X_raw["Universite Turu"] = train_X_raw["Universite Turu"].str.upper()
train_X_raw["Universite Turu"] = train_X_raw["Universite Turu"].fillna('DIGER')

test_X_raw["Universite Turu"] = test_X_raw["Universite Turu"].str.upper()
test_X_raw["Universite Turu"] = test_X_raw["Universite Turu"].fillna('DIGER')


### 'Burslu ise Burs Yuzdesi'(Binning), 'Burs Aliyor mu?(One Hot)

In [230]:
train_X_raw["Burs Aliyor mu?"] = train_X_raw["Burs Aliyor mu?"].str.title()
train_X_raw["Burs Aliyor mu?"] = train_X_raw["Burs Aliyor mu?"].fillna("Hayır")
test_X_raw["Burs Aliyor mu?"] = test_X_raw["Burs Aliyor mu?"].str.title()
test_X_raw["Burs Aliyor mu?"] = test_X_raw["Burs Aliyor mu?"].fillna("Hayır")

train_X_raw["Burs Aliyor mu?"] = train_X_raw["Burs Aliyor mu?"].map({'Evet': 1, 'Hayır': 0})
test_X_raw["Burs Aliyor mu?"] = test_X_raw["Burs Aliyor mu?"].map({'Evet': 1, 'Hayır': 0})

train_X_raw.loc[train_X_raw['Burs Aliyor mu?'] == 0, 'Burslu ise Burs Yuzdesi'] = int(0)
test_X_raw.loc[test_X_raw['Burs Aliyor mu?'] == 0, 'Burslu ise Burs Yuzdesi'] = int(0)
train_X_raw['Burslu ise Burs Yuzdesi'] = train_X_raw['Burslu ise Burs Yuzdesi'].fillna(int(0))
test_X_raw['Burslu ise Burs Yuzdesi'] = test_X_raw['Burslu ise Burs Yuzdesi'].fillna(int(0))

cut_labels = ['25altı', '25-49', '50-74', '75-99', '100']
cut_bins = [-1, 24.0, 49.0, 74.0, 99.0, 100.0]
#
train_X_raw['binned_burs_yuzde'] = pd.cut(train_X_raw['Burslu ise Burs Yuzdesi'], bins=cut_bins, labels=cut_labels)
test_X_raw['binned_burs_yuzde'] = pd.cut(test_X_raw['Burslu ise Burs Yuzdesi'], bins=cut_bins, labels=cut_labels)
train_X_raw.drop('Burslu ise Burs Yuzdesi', axis=1, inplace=True)
test_X_raw.drop('Burslu ise Burs Yuzdesi', axis=1, inplace=True)


### Bölüm(Frequnecy?)

In [231]:
train_X_raw['Bölüm'] = train_X_raw['Bölüm'].str.upper()
test_X_raw['Bölüm'] = test_X_raw['Bölüm'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Bölüm'] = train_X_raw['Bölüm'].apply(lambda x: str(x).replace(tr_char, en_char))
for tr_char, en_char in tr_chars.items():
    test_X_raw['Bölüm'] = test_X_raw['Bölüm'].apply(lambda x: str(x).replace(tr_char, en_char))

train_X_raw['Bölüm'] = train_X_raw['Bölüm'].fillna('DIGER')
train_X_raw['Bölüm'] = train_X_raw['Bölüm'].replace(r'^\s*$', 'DIGER', regex=True)
train_X_raw['Bölüm'] = train_X_raw['Bölüm'].replace('nan', 'DIGER')

test_X_raw['Bölüm'] = test_X_raw['Bölüm'].fillna('DIGER')
test_X_raw['Bölüm'] = test_X_raw['Bölüm'].replace(r'^\s*$', 'DIGER', regex=True)
test_X_raw['Bölüm'] = test_X_raw['Bölüm'].replace('nan', 'DIGER')

frequency = train_X_raw['Bölüm'].value_counts()
train_X_raw['encoded_bolum'] = train_X_raw['Bölüm'].map(frequency)
frequency = test_X_raw['Bölüm'].value_counts()
test_X_raw['encoded_bolum'] = test_X_raw['Bölüm'].map(frequency)

train_X_raw.drop('Bölüm', axis=1, inplace=True)
test_X_raw.drop('Bölüm', axis=1, inplace=True)

### Universite Kaçıncı Sınıf

In [232]:
replacements = {
    'HAZIRLIK': '0',
    'MEZUN': '7',
    'YUKSEK LISANS': '8',

    'TEZ': '6'
}

train_X_raw['Universite Kacinci Sinif'] = train_X_raw['Universite Kacinci Sinif'].str.upper()
test_X_raw['Universite Kacinci Sinif'] = test_X_raw['Universite Kacinci Sinif'].str.upper()

for tr_char, en_char in tr_chars.items():
    train_X_raw['Universite Kacinci Sinif'] = train_X_raw['Universite Kacinci Sinif'].str.replace(tr_char, en_char)
    test_X_raw['Universite Kacinci Sinif'] = test_X_raw['Universite Kacinci Sinif'].str.replace(tr_char, en_char)

for key, value in replacements.items():
    train_X_raw['Universite Kacinci Sinif'] = train_X_raw['Universite Kacinci Sinif'].str.replace(key, value)
    test_X_raw['Universite Kacinci Sinif'] = test_X_raw['Universite Kacinci Sinif'].str.replace(key, value)

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_X_raw['Universite Kacinci Sinif'] = imp.fit_transform(train_X_raw['Universite Kacinci Sinif'].values.reshape(-1,1))[:,0]
test_X_raw['Universite Kacinci Sinif'] = imp.transform(test_X_raw['Universite Kacinci Sinif'].values.reshape(-1,1))[:,0]

train_X_raw['Universite Kacinci Sinif'] = pd.to_numeric(train_X_raw['Universite Kacinci Sinif'], errors='coerce')
test_X_raw['Universite Kacinci Sinif'] = pd.to_numeric(test_X_raw['Universite Kacinci Sinif'], errors='coerce')

#print(train_X_raw['Universite Kacinci Sinif'].value_counts(dropna=False))
#print(test_X_raw['Universite Kacinci Sinif'].value_counts(dropna=False))

Universite Kacinci Sinif
2    22147
3    16956
4    13112
1    10260
0     1311
5      961
6      184
7      178
8       15
Name: count, dtype: int64
Universite Kacinci Sinif
2    3315
3    3039
4    2466
1    1803
0     233
5     164
6      29
Name: count, dtype: int64


### Universite Not Ortalamasi

In [233]:
replacements = {
    '4.0-3.5'    : '3.50 - 4.00',
    '4-3.5'      : '3.50 - 4.00',
    '3.50-3,6179': '3.50 - 4.00',
    '3.5 - 4.0'  : '3.50 - 4.00',

    '3.50-3'     : '3.00 - 3.50',
    '3.00 - 3.49': '3.00 - 3.50',

    '3.00-2.50'  : '2.50 - 3.00',
    '2.50 - 2.99': '2.50 - 3.00',
    '2.50 -3.00' : '2.50 - 3.00',

    '1.80 - 2.49': '2.50 VE ALTI',
    '2.00 - 2.50': '2.50 VE ALTI',
    '1.00 - 2.50': '2.50 VE ALTI',
    '0 - 1.79'   : '2.50 VE ALTI', 

    'ORTALAMA BULUNMUYOR': 'YOK',
    'HAZIRLIGIM': 'YOK',
    'NOT ORTALAMASI YOK': 'YOK',
    'nan': 'YOK'
}

replacements_test = {
    '2.50 - 2.99': '2.50 - 3.00',
    '3.00 - 3.49': '3.00 - 3.50',
    '1.80 - 2.49': '2.50 VE ALTI',
    '0 - 1.79'   : '2.50 VE ALTI',

    'HAZIRLIGIM' : 'YOK'
}

train_X_raw['Universite Not Ortalamasi'] = train_X_raw['Universite Not Ortalamasi'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Universite Not Ortalamasi'] = train_X_raw['Universite Not Ortalamasi'].apply(lambda x: str(x).replace(tr_char, en_char))
for key, value in replacements.items():
    train_X_raw['Universite Not Ortalamasi'] = train_X_raw['Universite Not Ortalamasi'].apply(lambda x: str(x).replace(key, value))

test_X_raw['Universite Not Ortalamasi'] = test_X_raw['Universite Not Ortalamasi'].str.upper()
for tr_char, en_char in tr_chars.items():
    test_X_raw['Universite Not Ortalamasi'] = test_X_raw['Universite Not Ortalamasi'].apply(lambda x: str(x).replace(tr_char, en_char))
for key, value in replacements_test.items():
    test_X_raw['Universite Not Ortalamasi'] = test_X_raw['Universite Not Ortalamasi'].apply(lambda x: str(x).replace(key, value))

value_counts = train_X_raw['Universite Not Ortalamasi'].value_counts()
top_5 = value_counts.nlargest(5)
train_X_raw['Universite Not Ortalamasi'] = train_X_raw['Universite Not Ortalamasi'].apply(lambda x: x if x in top_5.index else 'DIGER')

#train_X_raw['Universite Not Ortalamasi'].value_counts(dropna=False).to_csv("train.csv")
#test_X_raw['Universite Not Ortalamasi'].value_counts(dropna=False).to_csv("test.csv")

### 'Baska Bir Kurumdan Burs Aliyor mu?'

In [235]:
train_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'] = train_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'].fillna("Hayır")
test_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'] = test_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'].fillna("Hayır")

train_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'] = train_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'].map({'Evet': 1, 'Hayır': 0})
test_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'] = test_X_raw['Baska Bir Kurumdan Burs Aliyor mu?'].map({'Evet': 1, 'Hayır': 0})


### Anne Calisma Durumu, Anne Sektor

In [237]:
train_X_raw['Anne Calisma Durumu'] = train_X_raw['Anne Calisma Durumu'].fillna("bilinmiyor")
test_X_raw['Anne Calisma Durumu'] = test_X_raw['Anne Calisma Durumu'].fillna("bilinmiyor")

train_X_raw['Anne Sektor'] = train_X_raw['Anne Sektor'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Anne Sektor'] = train_X_raw['Anne Sektor'].apply(lambda x: str(x).replace(tr_char, en_char))
train_X_raw['Anne Sektor'] = train_X_raw['Anne Sektor'].apply(lambda x: x if x in ['OZEL SEKTOR', 'KAMU', 'DIGER'] else 'BELIRSIZ')

test_X_raw['Anne Sektor'] = test_X_raw['Anne Sektor'].str.upper()
for tr_char, en_char in tr_chars.items():
    test_X_raw['Anne Sektor'] = test_X_raw['Anne Sektor'].apply(lambda x: str(x).replace(tr_char, en_char))
test_X_raw['Anne Sektor'] = test_X_raw['Anne Sektor'].apply(lambda x: x if x in ['OZEL SEKTOR', 'KAMU', 'DIGER'] else 'BELIRSIZ')

#print(train_X_raw['Anne Sektor'].value_counts(dropna=False))

### Baba Calisma Durumu, Baba Sektor

In [238]:
train_X_raw['Baba Calisma Durumu'] = train_X_raw['Baba Calisma Durumu'].fillna("bilinmiyor")
test_X_raw['Baba Calisma Durumu'] = test_X_raw['Baba Calisma Durumu'].fillna("bilinmiyor")

train_X_raw['Baba Sektor'] = train_X_raw['Baba Sektor'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Baba Sektor'] = train_X_raw['Baba Sektor'].apply(lambda x: str(x).replace(tr_char, en_char))

test_X_raw['Baba Sektor'] = test_X_raw['Baba Sektor'].str.upper()
for tr_char, en_char in tr_chars.items():
    test_X_raw['Baba Sektor'] = test_X_raw['Baba Sektor'].apply(lambda x: str(x).replace(tr_char, en_char))

train_X_raw['Baba Sektor'] = train_X_raw['Baba Sektor'].apply(lambda x: x if x in ['OZEL SEKTOR', 'KAMU', 'DIGER'] else 'BELIRSIZ')
test_X_raw['Baba Sektor'] = test_X_raw['Baba Sektor'].apply(lambda x: x if x in ['OZEL SEKTOR', 'KAMU', 'DIGER'] else 'BELIRSIZ')

### Kardeş Sayısı

In [239]:
train_X_raw['Kardes Sayisi'] = pd.to_numeric(train_X_raw['Kardes Sayisi'], errors='coerce').fillna(0).astype(int)
test_X_raw['Kardes Sayisi'] = pd.to_numeric(test_X_raw['Kardes Sayisi'], errors='coerce').fillna(0).astype(int)

### Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?

In [240]:
train_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = train_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].fillna("Hayır")
test_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = test_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].fillna("Hayır")

train_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = train_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].map({'Evet': 1, 'Hayır': 0})
test_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'] = test_X_raw['Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?'].map({'Evet': 1, 'Hayır': 0})

### Profesyonel Bir Spor Daliyla Mesgul musunuz?

In [241]:
train_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = train_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].fillna("Hayır")
test_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = test_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].fillna("Hayır")

train_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = train_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].map({'Evet': 1, 'Hayır': 0})
test_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] = test_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'].map({'Evet': 1, 'Hayır': 0})

### Spor Dalindaki Rolunuz Nedir?

In [242]:
replacements = {
    'KAPTAN / LİDER': 'KAPTAN',
    'Lider/Kaptan'  : 'KAPTAN',
    'Kaptan'        : 'KAPTAN',

    'Takım Oyuncusu': 'OYUNCU',
    'Bireysel'      : 'OYUNCU',

    'DİĞER'         : 'DIGER',
    'Diğer'         : 'DIGER',
}

replacements_test = {
    'Bireysel Spor' : 'OYUNCU',
    'Takım Oyuncusu': 'OYUNCU',

    'Lider/Kaptan'  : 'KAPTAN',
    'Diğer'         : 'DIGER'
}

for key, value in replacements.items():
    train_X_raw['Spor Dalindaki Rolunuz Nedir?'] = train_X_raw['Spor Dalindaki Rolunuz Nedir?'].apply(lambda x: str(x).replace(key, value))
for key, value in replacements_test.items():
    test_X_raw['Spor Dalindaki Rolunuz Nedir?'] = test_X_raw['Spor Dalindaki Rolunuz Nedir?'].apply(lambda x: str(x).replace(key, value))

train_X_raw.loc[train_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] == 'Hayır', 'Spor Dalindaki Rolunuz Nedir?'] = 'nan'
test_X_raw.loc[test_X_raw['Profesyonel Bir Spor Daliyla Mesgul musunuz?'] == 'Hayır', 'Spor Dalindaki Rolunuz Nedir?'] = 'nan'
train_X_raw['Spor Dalindaki Rolunuz Nedir?'] = train_X_raw['Spor Dalindaki Rolunuz Nedir?'].apply(lambda x: str(x).replace('nan', 'YOK'))
test_X_raw['Spor Dalindaki Rolunuz Nedir?'] = test_X_raw['Spor Dalindaki Rolunuz Nedir?'].apply(lambda x: str(x).replace('nan', 'YOK'))

train_X_raw['Spor Dalindaki Rolunuz Nedir?'] = train_X_raw['Spor Dalindaki Rolunuz Nedir?'].map({'KAPTAN': 3,'OYUNCU': 2,'DIGER': 1, 'YOK': 0})
test_X_raw['Spor Dalindaki Rolunuz Nedir?'] = test_X_raw['Spor Dalindaki Rolunuz Nedir?'].map({'KAPTAN': 3,'OYUNCU': 2,'DIGER': 1, 'YOK': 0})

#frequency = train_X_raw['Spor Dalindaki Rolunuz Nedir?'].value_counts()
#train_X_raw["encoded_spor_rol"] = train_X_raw['Spor Dalindaki Rolunuz Nedir?'].map(frequency)
#frequency = test_X_raw['Spor Dalindaki Rolunuz Nedir?'].value_counts()
#test_X_raw["encoded_spor_rol"] = test_X_raw['Spor Dalindaki Rolunuz Nedir?'].map(frequency)

#train_X_raw.drop('Spor Dalindaki Rolunuz Nedir?', axis=1, inplace=True)
#test_X_raw.drop('Spor Dalindaki Rolunuz Nedir?', axis=1, inplace=True)
#train_X_raw['Spor Dalindaki Rolunuz Nedir?'].value_counts(dropna=False)

### 'Ingilizce Biliyor musunuz?', 'Ingilizce Seviyeniz?'

In [243]:
train_X_raw['Ingilizce Biliyor musunuz?'] = train_X_raw['Ingilizce Biliyor musunuz?'].fillna("Hayır")
test_X_raw['Ingilizce Biliyor musunuz?'] = test_X_raw['Ingilizce Biliyor musunuz?'].fillna("Hayır")

train_X_raw['Ingilizce Biliyor musunuz?'] = train_X_raw['Ingilizce Biliyor musunuz?'].map({'Evet': 1, 'Hayır': 0})
test_X_raw['Ingilizce Biliyor musunuz?'] = test_X_raw['Ingilizce Biliyor musunuz?'].map({'Evet': 1, 'Hayır': 0})

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_X_raw['Ingilizce Seviyeniz?'] = imp.fit_transform(train_X_raw['Ingilizce Seviyeniz?'].values.reshape(-1,1))[:,0]
test_X_raw['Ingilizce Seviyeniz?'] = imp.transform(test_X_raw['Ingilizce Seviyeniz?'].values.reshape(-1,1))[:,0]
train_X_raw.loc[train_X_raw['Ingilizce Seviyeniz?'] != "0", 'Ingilizce Biliyor musunuz?'] = 1
test_X_raw.loc[test_X_raw['Ingilizce Seviyeniz?'] != "0", 'Ingilizce Biliyor musunuz?'] = 1

train_X_raw['Ingilizce Seviyeniz?'] = train_X_raw['Ingilizce Seviyeniz?'].map({'İleri': 3,'Orta': 2,'Başlangıç': 1, '0': 0})
test_X_raw['Ingilizce Seviyeniz?'] = test_X_raw['Ingilizce Seviyeniz?'].map({'İleri': 3,'Orta': 2,'Başlangıç': 1, '0': 0})
#train_X_raw['Ingilizce Seviyeniz?'].value_counts(dropna=False)

### Aktif olarak bir STK üyesi misiniz?

In [244]:
train_X_raw['Aktif olarak bir STK üyesi misiniz?'] = train_X_raw['Aktif olarak bir STK üyesi misiniz?'].fillna("Hayır")
test_X_raw['Aktif olarak bir STK üyesi misiniz?'] = test_X_raw['Aktif olarak bir STK üyesi misiniz?'].fillna("Hayır")
train_X_raw['Aktif olarak bir STK üyesi misiniz?'] = train_X_raw['Aktif olarak bir STK üyesi misiniz?'].map({'Evet': 1, 'Hayır': 0})
test_X_raw['Aktif olarak bir STK üyesi misiniz?'] = test_X_raw['Aktif olarak bir STK üyesi misiniz?'].map({'Evet': 1, 'Hayır': 0})

### Stk Projesine Katildiniz Mi?

In [245]:
train_X_raw['Stk Projesine Katildiniz Mi?'] = train_X_raw['Stk Projesine Katildiniz Mi?'].fillna('Hayır')
test_X_raw['Stk Projesine Katildiniz Mi?'] = test_X_raw['Stk Projesine Katildiniz Mi?'].fillna('Hayır')

train_X_raw['Stk Projesine Katildiniz Mi?'] = train_X_raw['Stk Projesine Katildiniz Mi?'].map({'Evet': 1, 'Hayır': 0})
test_X_raw['Stk Projesine Katildiniz Mi?'] = test_X_raw['Stk Projesine Katildiniz Mi?'].map({'Evet': 1, 'Hayır': 0})

### Uye Oldugunuz Kulubun Ismi

In [246]:
#train_X_raw['Uye Oldugunuz Kulubun Ismi'].value_counts(dropna=False)
#
#train_X_raw['Uye Oldugunuz Kulubun Ismi'] = train_X_raw['Uye Oldugunuz Kulubun Ismi'].astype(str).str.upper()
#test_X_raw['Uye Oldugunuz Kulubun Ismi'] = test_X_raw['Uye Oldugunuz Kulubun Ismi'].astype(str).str.upper()
#for tr_char, en_char in tr_chars.items():
#    train_X_raw['Uye Oldugunuz Kulubun Ismi'] = train_X_raw['Uye Oldugunuz Kulubun Ismi'].apply(lambda x: str(x).replace(tr_char, en_char))
#for tr_char, en_char in tr_chars.items():
#    test_X_raw['Uye Oldugunuz Kulubun Ismi'] = test_X_raw['Uye Oldugunuz Kulubun Ismi'].apply(lambda x: str(x).replace(tr_char, en_char))
#
#value_counts = train_X_raw['Uye Oldugunuz Kulubun Ismi'].value_counts()
#top_5 = value_counts.nlargest(5)
#train_X_raw['Uye Oldugunuz Kulubun Ismi'] = train_X_raw['Uye Oldugunuz Kulubun Ismi'].apply(lambda x: x if x in top_5.index else 'DIGER')

### Girisimcilikle Ilgili Deneyiminiz Var Mi?

In [247]:
train_X_raw['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = train_X_raw['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].fillna('Hayır')
test_X_raw['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = test_X_raw['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].fillna('Hayır')

train_X_raw['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = train_X_raw['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].map({'Evet': 1, 'Hayır': 0})
test_X_raw['Girisimcilikle Ilgili Deneyiminiz Var Mi?'] = test_X_raw['Girisimcilikle Ilgili Deneyiminiz Var Mi?'].map({'Evet': 1, 'Hayır': 0})

### "Hangi STK'nin Uyesisiniz?"

In [248]:
#replacements = {
#    'TURK KIZILAYI'    : 'KIZILAY',
#    'GENC KIZILAY'     : 'KIZILAY',
#    'KIZILAY GONULLUSU': 'KIZILAY',
#    'TURK KIZILAY'     : 'KIZILAY',
#    'KIZILAY KULUBU'   : 'KIZILAY',
#    'KIZILA'           : 'KIZILAY',
#    'KIZILAYY'         : 'KIZILAY',
#
#    'TEMA VAKFI'       : 'TEMA',
#    'GENC TEMA'        : 'TEMA',
#    'TEMA VAKFINA UYEYIM': 'TEMA',
#    'TEMA GONULLUSUYUM.' : 'TEMA',
#    'TEMA VAKFI GONULLUSUYUM.': 'TEMA',
#    'TEMA VAKFIN'      : 'TEMA',
#
#    'TOPLUM GONULLULERI': 'TOG',
#    'TOPLUM GONULLULERI VAKFI': 'TOG',
#    'TOG VAKFI'         : 'TOG',
#    'TOPLUM GONULLULERI VAKFI (TOG)': 'TOG',
#    'TOPLUM GONULLULERI(TOG)': 'TOG',
#    'TOPLUM GONULLULERI VAKFITOG': 'TOG',
#
#    'TURKIYE EGITIM GONULLULERI VAKFI': 'TEGV',
#
#    'YOUNG GURU ACADEMY': 'YGA',
#
#    'CAGDAS YASAMI DESTEKLEME DERNEGI': 'CYDD'
#}
#
#train_X_raw["Hangi STK'nin Uyesisiniz?"] = train_X_raw["Hangi STK'nin Uyesisiniz?"].fillna("YOK")
#test_X_raw["Hangi STK'nin Uyesisiniz?"] = test_X_raw["Hangi STK'nin Uyesisiniz?"].fillna("YOK")
#
#train_X_raw["Hangi STK'nin Uyesisiniz?"] = train_X_raw["Hangi STK'nin Uyesisiniz?"].str.upper()
#train_X_raw["Hangi STK'nin Uyesisiniz?"] = train_X_raw["Hangi STK'nin Uyesisiniz?"].str.strip()
#train_X_raw["Hangi STK'nin Uyesisiniz?"] = train_X_raw["Hangi STK'nin Uyesisiniz?"].str.replace(r'^"|"$', '', regex=True)
#for tr_char, en_char in tr_chars.items():
#    train_X_raw["Hangi STK'nin Uyesisiniz?"] = train_X_raw["Hangi STK'nin Uyesisiniz?"].apply(lambda x: str(x).replace(tr_char, en_char))
#for key, value in replacements.items():
#    train_X_raw["Hangi STK'nin Uyesisiniz?"] = train_X_raw["Hangi STK'nin Uyesisiniz?"].apply(lambda x: str(x).replace(key, value))
#
#test_X_raw["Hangi STK'nin Uyesisiniz?"] = test_X_raw["Hangi STK'nin Uyesisiniz?"].str.upper()
#test_X_raw["Hangi STK'nin Uyesisiniz?"] = test_X_raw["Hangi STK'nin Uyesisiniz?"].str.strip()
#test_X_raw["Hangi STK'nin Uyesisiniz?"] = test_X_raw["Hangi STK'nin Uyesisiniz?"].str.replace(r'^"|"$', '', regex=True)
#for tr_char, en_char in tr_chars.items():
#    test_X_raw["Hangi STK'nin Uyesisiniz?"] = test_X_raw["Hangi STK'nin Uyesisiniz?"].apply(lambda x: str(x).replace(tr_char, en_char))
#
#train_X_raw["Hangi STK'nin Uyesisiniz?"] = train_X_raw["Hangi STK'nin Uyesisiniz?"].str.replace("-", "YOK")
#test_X_raw["Hangi STK'nin Uyesisiniz?"] = test_X_raw["Hangi STK'nin Uyesisiniz?"].str.replace("-", "YOK")
#
#
#train_X_raw["Hangi STK'nin Uyesisiniz?"].value_counts(dropna=False).to_csv("a.csv")

#train_X_raw["Hangi STK'nin Uyesisiniz?"].value_counts(dropna=False).to_csv("a.csv")

### 'Lise Turu'

In [249]:
replacements = {
    'OZEL LISESI': 'OZEL',
    'OZEL LISE': 'OZEL',
    
    'MESLEK LISESI': 'DEVLET',
    'FEN LISESI': 'DEVLET',
    'DUZ LISE': 'DEVLET',
    'ANADOLU LISESI': 'DEVLET',
    'MESLEK': 'DEVLET',
    'IMAM HATIP LISESI': 'DEVLET',

    'BILINMIYOR': 'DIGER'
}

train_X_raw['Lise Turu'] = train_X_raw['Lise Turu'].fillna("DIGER")
test_X_raw['Lise Turu'] = test_X_raw['Lise Turu'].fillna("DIGER")

train_X_raw['Lise Turu'] = train_X_raw['Lise Turu'].str.upper()
for tr_char, en_char in tr_chars.items():
    train_X_raw['Lise Turu'] = train_X_raw['Lise Turu'].apply(lambda x: str(x).replace(tr_char, en_char))
for key, value in replacements.items():
    train_X_raw['Lise Turu'] = train_X_raw['Lise Turu'].apply(lambda x: str(x).replace(key, value))

test_X_raw['Lise Turu'] = test_X_raw['Lise Turu'].str.upper()
for tr_char, en_char in tr_chars.items():
    test_X_raw['Lise Turu'] = test_X_raw['Lise Turu'].apply(lambda x: str(x).replace(tr_char, en_char))

### Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?

In [250]:
test_X_raw['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].value_counts(dropna=False).to_csv("a.csv")

In [251]:

train_X_raw['encoded_girisimcilik_deneyim'] = train_X_raw['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].str.len()
test_X_raw['encoded_girisimcilik_deneyim'] = test_X_raw['Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'].str.len()
train_X_raw.drop('Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?', axis=1, inplace=True)
test_X_raw.drop('Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?', axis=1, inplace=True)

## One Hot Encoding

In [252]:
encode_features = ['Cinsiyet', 'Universite Turu', 'binned_burs_yuzde', 'Lise Turu',
                   'Universite Not Ortalamasi', 'Anne Calisma Durumu', 
                   'Baba Calisma Durumu', 'Baba Sektor' ,'Anne Sektor', 
                   ]

encoder = OneHotEncoder(sparse_output=False)

encoded_train = encoder.fit_transform(train_X_raw[encode_features])
categories = encoder.get_feature_names_out(encode_features)
encoded_df_train = pd.DataFrame(encoded_train, columns=categories)

encoded_test = encoder.transform(test_X_raw[encode_features])
encoded_df_test = pd.DataFrame(encoded_test, columns=categories)

train_X_raw = pd.concat([train_X_raw, encoded_df_train], axis=1)
test_X_raw = pd.concat([test_X_raw, encoded_df_test], axis=1)

train_X_raw.drop(encode_features, axis=1, inplace=True)
test_X_raw.drop(encode_features, axis=1, inplace=True)


### Corr Matrix

In [253]:

corr_matrix = train_X_raw.corr()
cor_target = abs(corr_matrix["Degerlendirme Puani"])
relevant_features = cor_target
relevant_features = relevant_features.sort_values(ascending=False)
relevant_features.to_csv("rev.csv")

In [254]:
train_X_raw = train_X_raw.drop('Degerlendirme Puani', axis=1)

# Eğitim

In [255]:
train_X = train_X_raw
train_y = train_y_raw

test_X = test_X_raw

X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2 ,random_state=42)


model = RandomForestRegressor(n_jobs=-1 ,random_state=42)
model_xg = XGBRegressor(objective = 'reg:squarederror',
                        subsample = 1.0, 
                        reg_lambda = 3, 
                        reg_alpha =  0.1, 
                        n_estimators = 100, 
                        min_child_weight = 5, 
                        max_depth = 10, 
                        learning_rate = 0.1, 
                        gamma = 0.1, 
                        colsample_bytree = 0.8,
                        nthread = -1) 

#param_dist = {
#    'n_estimators': [100],
#    'learning_rate': [0.01, 0.1, 0.2],
#    'max_depth': [10],
#    'min_child_weight': [1, 3, 5],
#    'subsample': [0.6, 0.8, 1.0],
#    'colsample_bytree': [0.6, 0.8, 1.0],
#    'gamma': [0, 0.1, 0.3, 0.5],
#    'reg_alpha': [0, 0.01, 0.1, 1],
#    'reg_lambda': [1, 1.5, 2, 3]
#}


#random_search = RandomizedSearchCV(model_xg, param_distributions=param_dist,
#                                   n_iter=50, scoring='neg_mean_squared_error',
#                                   cv=3, verbose=1, n_jobs=-1, random_state=42)
#random_search.fit(X_train, y_train)
model.fit(X_train, y_train)
model_xg.fit(X_train, y_train)

#print("Best Parameters: ", random_search.best_params_)

#best_xgb = random_search.best_estimator_

y_pred_xg = model_xg.predict(X_val)
y_pred_xg_train = model_xg.predict(X_train)
y_pred = model.predict(X_val)
y_pred_train = model.predict(X_train)

# Step 9: Calculate evaluation metrics for regression
rmse_xg = np.sqrt(((y_val - y_pred_xg) ** 2).mean())
rmse_xg_train = np.sqrt(((y_train - y_pred_xg_train) ** 2).mean())
rmse = np.sqrt(((y_val - y_pred) ** 2).mean())
rmse_train = np.sqrt(((y_train - y_pred_train) ** 2).mean())

print(f"Train xg rmse: {rmse_xg_train}")
print(f"Val xg rmse: {rmse_xg}")
print(f"Train rf rmse: {rmse_train}")
print(f"Val rf rmse: {rmse}")


Train xg rmse: 4.196656078131464
Val xg rmse: 5.67685595935996
Train rf rmse: 2.2405468077550936
Val rf rmse: 5.982898420376664


In [257]:
predictions = model_xg.predict(test_X)
output = pd.DataFrame({'id': test_raw['id'],
                       'Degerlendirme Puani': predictions})
output.to_csv('submission.csv', index=False)